In [49]:
# Парсим первую страницу
import requests
import re
import json
from bs4 import BeautifulSoup
from fake_headers import Headers

def get_headers():
    return Headers(os='win', browser='chrome').generate()

response = requests.get('https://spb.hh.ru/search/vacancy?text=python&area=1&area=2', headers=get_headers())
html_data = response.text
soup = BeautifulSoup(html_data, features = 'lxml')

main_page = soup.find('div', id = 'a11y-main-content') # Получаем главную страницу 
vacancy_blocks = main_page.find_all('div', class_ = 'vacancy-card--z_UXteNo7bRGzxWVcL7y font-inter') # Получаем отдельные блоки с вакансиями

# Задаем регулярные выражения для поиска слов в описании вакансии
pattern1 = re.compile(r'Django', re.IGNORECASE)
pattern2 = re.compile(r'Flask', re.IGNORECASE)

result = []
without_description=[] # Список на ссылки тех вакансий, где нет блока с описанием
for elem in vacancy_blocks: # Цикл по каждому блоку
    a_tag = elem.find('a', class_ = 'bloko-link') # Получаем элемент с тэгом "а"
    absolute_link = a_tag['href'] # Получаем саму ссылку на вакансию

    article_soup = BeautifulSoup(requests.get(absolute_link, headers=get_headers()).text) # Переходим по ссылке
    salary = article_soup.find('span', class_ = 'magritte-text___pbpft_3-0-9 magritte-text_style-primary___AQ7MW_3-0-9 magritte-text_typography-label-1-regular___pi3R-_3-0-9').text.strip() # Получаем зарплату
    cleaned_salary = salary.replace('\xa0', ' ') # Удаляем ненужные символы в строке
    # Получаем название компании и проверяем указана она или нет
    name_of_company = article_soup.find('span', class_ = 'bloko-header-section-2 bloko-header-section-2_lite')
    if name_of_company:
        name_of_company = name_of_company.get_text(strip=True)
    else:
        name_of_company = "Компания не указана"
    # Получаем город
    city = article_soup.find('p', {'data-qa': 'vacancy-view-location'})
    if city:
        city = city.get_text(strip=True)
    else:
        city = article_soup.find('span', {'data-qa': 'vacancy-view-link-location-text'}).get_text()
    
    # Получаем описание и проверяем на существование 
    vacancy_description = article_soup.find('div', class_ = 'vacancy-description')
    if not vacancy_description:
        without_description.append(absolute_link)
        continue
    vacancy_description = vacancy_description.text
    match1 = pattern1.search(vacancy_description)
    match2 = pattern2.search(vacancy_description)
    if match1 and match2: # Проверка на то, чтобы оба слова были в описании
        result.append({
            'link': absolute_link,
            'salary': cleaned_salary,
            'name_of_company': name_of_company,
            'city': city
        })
        

print(result)
print(without_description) 

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

[{'link': 'https://spb.hh.ru/vacancy/101525768?query=python&hhtmFrom=vacancy_search_list', 'salary': 'до 470 000 ₽ на руки', 'name_of_company': 'Точка', 'city': 'Москва'}, {'link': 'https://spb.hh.ru/vacancy/101676685?query=python&hhtmFrom=vacancy_search_list', 'salary': 'Уровень дохода не указан', 'name_of_company': 'AldanDev', 'city': 'Москва'}, {'link': 'https://spb.hh.ru/vacancy/101864904?query=python&hhtmFrom=vacancy_search_list', 'salary': 'Уровень дохода не указан', 'name_of_company': 'ОООИНФОМЕДИА', 'city': 'Москва, Цветной бульвар, 25с5'}, {'link': 'https://spb.hh.ru/vacancy/102660978?query=python&hhtmFrom=vacancy_search_list', 'salary': 'Уровень дохода не указан', 'name_of_company': 'Admire', 'city': 'Москва'}]
[]
